# MAIA Demo

#### Many of MAIA's experiments are available in the [experiment browser](https://multimodal-interpretability.csail.mit.edu/maia/experiment-browser/) ####

In [ ]:
%load_ext autoreload
%autoreload 2
# TODO - Convert to Demo

In [2]:
import sys
import os
sys.path.append(os.path.abspath('VisDiff'))

In [3]:
!export CUDA_VISIBLE_DEVICES=1
import os
from IPython import embed

import openai
from dotenv import load_dotenv

# Some imports require api key to be set ######
# Load environment variables
load_dotenv()

# Load OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORGANIZATION")
print(type(openai.api_key))
###############################################

from maia_api import System, Tools
from utils.DatasetExemplars import DatasetExemplars
from utils.main_utils import generate_save_path, create_unit_config
from utils.CodeAgent import CodeAgent

<class 'str'>


Matplotlib created a temporary cache directory at /tmp/matplotlib-7nj5wq72 because the default path (/afs/csail.mit.edu/u/j/jaketouchet/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
INFO:matplotlib.font_manager:generated new fontManager


In [4]:
maia = 'gpt-4o'
task = 'single'
n_exemplars = 15
model1 = "finetune_resnet_gelu"
model2 = "gradnorm_resnet_gelu"
layer = "layer4"
neuron = 20
images_per_prompt = 1
path2save = './results'
path2prompts = './prompts'
path2exemplars = './exemplars'
device = 1
text2image = 'sd'
debug = True

unit_config_name = f"{model1}_{model2}_{layer}_{neuron}"
unit_config = create_unit_config(model1, model2, layer, neuron)

path2save = generate_save_path(path2save, maia, unit_config_name)
print(path2save)
os.makedirs(path2save, exist_ok=True)

./results/gpt-4o/finetune_resnet_gelu_gradnorm_resnet_gelu_layer4_20


In [5]:
code_agent = CodeAgent(
            model_name=maia,
            prompt_path=path2prompts,
            api_prompt_name="api.txt",
            user_prompt_name=f"user_{task}.txt",
            overload_prompt_name="final.txt",
            end_experiment_token="[FINAL]",
            max_round_count=25,
            debug=debug
        )
net_dissect = DatasetExemplars(path2exemplars, n_exemplars, path2save, unit_config)
system = System(unit_config, net_dissect.thresholds, device)
tools = Tools(path2save, device, maia, system, net_dissect, images_per_prompt=images_per_prompt, text2image_model_name=text2image, image2text_model_name=maia)

print(tools.html_path)

In [25]:
code_agent.run_experiment(system, tools, save_html=True)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Let's begin by first exploring the dataset exemplars of both units [0, 1]. This will give us initial insights into the types of images that maximally activate these units. We will use the `dataset_exemplars` method from the `Tools` class. 

```python
# Retrieve and display the activations and exemplar images for units 0 and 1
unit_ids = [0, 1]
exemplar_data = tools.dataset_exemplars(unit_ids)
for i in range(len(exemplar_data)):
    tools.display(f"unit {unit_ids[i]}: ")
    for activation, masked_image in exemplar_data[i]:
        tools.display(masked_image, activation)
```


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ANALYSIS]
From the dataset exemplars, we can see that Unit 0 has high activations for images with patterns or textures related to snakes and, to a lesser extent, other animals like dogs and meerkats. On the other hand, Unit 1 seems to react strongly to images containing meerkats and similar shapes like the yellow banana.

[HYPOTHESIS LIST]
UNIT 0:
- Snake patterns and textures (high contrast lines, elongated shapes) #M
- Animal faces #M
- Curved/elongated objects #L

UNIT 1:
- Meerkat faces #H
- Yellow and green elongated shapes #M
- Animal faces #M

To further investigate the differences, we can create and synthesize images using text-to-image prompts that capture specific aspects of these hypotheses. This will help confirm or reject our initial observations about the differences in activations between the two units.

Let's start by testing our hypothesis about animal faces and patterns in these synthesized images.

[CODE]

```python
# Generate images from prompts specifically design

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

[ANALYSIS]
From the results of our most recent experiment, we can make the following observations:

- **Image 0 (Yellow snake on a branch)**:
  - Unit 0: Activation 2.17
  - Unit 1: Activation 6.82

  Both units responded, but Unit 1 activated significantly more. This indicates a preference for elongated objects, but the context might matter more for Unit 1.

- **Image 1 (Close-up of a yellow banana)**:
  - Unit 0: Activation 3.13
  - Unit 1: Activation 7.57

  Both units responded, with Unit 1 showing a stronger response. The shape and color might be a significant factor here.

- **Image 2 (Yellow pencil on a blue notebook)**:
  - Unit 0: Activation -0.0
  - Unit 1: Activation -2.1612e-05

  Neither unit showed notable activation, suggesting the elongated shape and color need a more defined context.

- **Image 3 (Meerkat with a yellow hat)**:
  - Unit 0: Activation 0.24
  - Unit 1: Activation 6.86

  Unit 1 strongly activated again, aligning with the hypothesis that it is highly sensi

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ANALYSIS]
From the results of this experiment with various yellow elongated and ambiguous shapes, we can make the following observations:

- **Image 0 (Yellow string on a white background)**:
  - Unit 0: Activation 1.50
  - Unit 1: Activation 2.74

  Both units activated, with Unit 1 showing a stronger response. This indicates that both units are sensitive to elongated yellow shapes, but Unit 1 is more sensitive.

- **Image 1 (Yellow rattan rope)**:
  - Unit 0: Activation -0.0
  - Unit 1: Activation 3.33

  Unit 1 strongly activated, but Unit 0 did not activate at all. This suggests Unit 1's sensitivity to woven or intricate yellow shapes.

- **Image 2 (Yellow hose coiled on the ground)**:
  - Unit 0: Activation 0.08
  - Unit 1: Activation 3.94

  Unit 1 showed a much stronger activation, reinforcing the sensitivity to coiled or complex elongated shapes in yellow context for Unit 1.

- **Image 3 (Yellow plank on brown soil)**:
  - Unit 0: Activation 0.74
  - Unit 1: Activation 1.20

 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ANALYSIS]
From the results of this experiment with various animal faces, we can make the following observations:

- **Image 0 (Close-up of a dog face)**:
  - Unit 0: Activation -0.0
  - Unit 1: Activation 0.194

  Neither unit activated significantly, but Unit 1 had a slightly higher activation.

- **Image 1 (Close-up of a cat face)**:
  - Unit 0: Activation -0.0
  - Unit 1: Activation 0.619

  Similar to the dog, neither unit activated significantly, but Unit 1 had a higher activation.

- **Image 2 (Close-up of a bird face)**:
  - Unit 0: Activation 0.728
  - Unit 1: Activation 3.915

  Both units activated, with Unit 1 showing a stronger response. This suggests some sensitivity to bird faces.

- **Image 3 (Close-up of a meerkat face with whiskers)**:
  - Unit 0: Activation 4.746
  - Unit 1: Activation 4.041

  Both units showed strong activation, with Unit 0 showing a slightly stronger response.

- **Image 4 (Close-up of a snake's head)**:
  - Unit 0: Activation -0.0
  - Unit 1: Act

In [ ]:
exemplar_data = tools.dataset_exemplars([1], system)
activations = [activation for activation, _ in exemplar_data[0]]
hypothesis = "Unit 1 is more selective to specific breeds of dogs, particularly those with long fur and fluffy grooming styles."
#hypothesis = "Unit 1 is more selective to specific breeds of dogs, particularly those with long fur and fluffy grooming styles."
context = f"Top 15 activations: {activations}"
result = tools.test_hypothesis(hypothesis, context, debug=True)
print(debug)

In [ ]:
exemplar_data = tools.dataset_exemplars([0, 1], system)
activations_0 = [activation for activation, _ in exemplar_data[0]]
activations_1 = [activation for activation, _ in exemplar_data[0]]
hypothesis = "Unit 1 is more selective to specific breeds of dogs, particularly those with long fur and fluffy grooming styles."
#hypothesis = "Unit 1 is more selective to specific breeds of dogs, particularly those with long fur and fluffy grooming styles."
context = f"Top 15 activations for unit 0: {activations_0}\n"
context += f"Top 15 activations for unit 1: {activations_1}"
result = tools.test_hypothesis(hypothesis, context, debug=True)
print(debug)

In [ ]:
tools.experiment_log = []
tools.update_experiment_log(role='system', type="text", type_content=maia_api) # update the experiment log with the system prompt
tools.update_experiment_log(role='user', type="text", type_content=user_query) # update the experiment log with the user prompt

j = 0
for i in range(20):
    print(i)
    maia_experiment = ask_agent(maia,tools.experiment_log) # ask maia for the next experiment given the results log to the experiment log (in the first round, the experiment log contains only the system prompt (maia api) and the user prompt (the query))
    tools.update_experiment_log(role='maia', type="text", type_content=str(maia_experiment)) # update the experiment log with maia's response (str casting is for exceptions)
    tools.generate_html() # generate the html file to visualize the experiment log
    if "[Difference]" in maia_experiment: break # stop the experiment if the response contains the final description. "[DESCRIPTION]" is the stopping signal.  
    experiment_output = experiment_env.execute_experiment(maia_experiment)
    if experiment_output != "":
        tools.update_experiment_log(role='user', type="text", type_content=experiment_output)

In [ ]:
tools.experiment_log = []
print(tools.visdiff(system, mode="OBJECTS"))
print("Done")

In [ ]:
tools.experiment_log = []
unit_ids = [0]
exemplar_data = tools.dataset_exemplars(unit_ids, system)
exemplars = [exemplar for _, exemplar in exemplar_data[0]]
print(tools.summarize_images(exemplars, debug=True))

In [ ]:
for log in tools.experiment_log:
    print(log)

In [ ]:
print(ask_agent("gpt-4o",[tools.experiment_log[-1]]))